In [1]:
!pip3 install sparqlwrapper
!pip3 install datasets

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os
import sys
import json

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import datasets

ROOT = os.getcwd().replace('ml/nlp', '')
print(ROOT)

/Users/anushmutyala/Documents/NLP-and-Clustering-for-Topic-Modeelling/


In [ ]:
from torch.utils.data import Dataset
from torchtext.data.functional import custom_replace, simple_space_split
from itertools import chain

def parse_xml(path):
    with open(path, 'r') as train_corpus: 
        def format_tag(line):
            all_starts = [idx for idx, item in enumerate(line) if '<passage>' in item]
            all_ends = [idx for idx, item in enumerate(line) if '</passage>' in item]
            return [[all_starts[idx], all_ends[idx]] for idx in range(len(all_starts))]
        data = train_corpus.readlines()

        ind = format_tag(data)
        return data, ind

def get_raw_txt(path):
    raw, indices = parse_xml(path)

    all_entries = [raw[indices[idx][0]+1:indices[idx][1]] for idx in range(len(indices))]
    all_txt = []

    for x, entry in enumerate(all_entries):
        empty = []
        for idx, item in enumerate(entry):
            if '<text>' in str(item):
                add_txt = entry[idx].replace('<text>', '')
                add_txt = add_txt.replace('</text>', '')
                add_txt = add_txt.replace('\n', '')
                empty.append(add_txt)
            else:
                pass
        all_txt.append(''.join(empty))
    return all_txt

text_db = get_raw_txt(ROOT + 'ml/data/nlm-chem-corpus/ALL/1253656_v1.xml')

custom_replace_transform = custom_replace([(r'\n', ''), (r'<', ''), (r'>', ''), (r'\t', '')])
preprocess = lambda x: (simple_space_split(custom_replace_transform(x)))

class PaperTextDataset(Dataset):
    def __init__ (self):
        self.data = {}
        self.embeddings = []
    
    def add_dataset(self, name, path, transformation):
        transformed_data = []
        for file in os.listdir(path):
            try:
                processed_txt = list(preprocess(transformation(path+'/'+file)))
                transformed_data.append(list(chain.from_iterable(processed_txt)))
            except:
                print(path+'/'+file)
                break

        # transformed_data = [preprocess(transformation(file)) for file in os.listdir(path)]
        self.data[name] = transformed_data
        return f"Added {name} to dataset."

    def __len__(self):
        return len(self.data)

    def __getitem__(self, name, idx):
        sample = self.data[name][idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def embed(self, tokenizer, model):
        self.embeddings = []
        for _, data in self.data.items():
            for text in data:
                inputs = torch.tensor(tokenizer.encode(text[0])).unsqueeze(0)
                outputs = model(inputs)
                self.embeddings.append(outputs)
        return self.embeddings


db = PaperTextDataset()
db.add_dataset('nlm-chem-corpus', ROOT+'ml/data/nlm-chem-corpus/ALL', get_raw_txt)

'Added nlm-chem-corpus to dataset.'

In [ ]:
raw_txt = db.data['nlm-chem-corpus']
print(len(raw_txt))

raw_txt_folded = [[i] for i in raw_txt]

docs_df = pd.DataFrame(raw_txt_folded, columns=["Doc"])
# docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_df.head()
# docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})


150


,Doc,Doc_ID
0,"[Occupational, exposure, to, chlorinated, solv...",0
1,"[Prophylactic, cannabinoid, administration, bl...",1
2,"[The, green, tea, extract, epigallocatechin-3-...",2
3,"[GABAergic, inhibition, is, weakened, or, conv...",3
4,"[Marker-Assisted, Selection, for, Recognizing,...",4


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import os
import sys

# from torch_geometric.data import Data, DataLoader
# from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool
# from torch_geometric.utils import to_networkx

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from datasets import load_dataset

ROOT = os.getcwd().replace('ml/nlp', '')
print(ROOT)

dataset = load_dataset('ncbi_disease')
df_ncbi = pd.DataFrame(dataset['train'])

df_ncbi.head()

from torch.utils.data import Dataset
from torchtext.data.functional import custom_replace, simple_space_split
from itertools import chain
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

def parse_xml(path):
    with open(path, 'r') as train_corpus: 
        def format_tag(line):
            all_starts = [idx for idx, item in enumerate(line) if '<passage>' in item]
            all_ends = [idx for idx, item in enumerate(line) if '</passage>' in item]
            return [[all_starts[idx], all_ends[idx]] for idx in range(len(all_starts))]
        data = train_corpus.readlines()

        ind = format_tag(data)
        return data, ind

def get_raw_txt(path):
    raw, indices = parse_xml(path)

    all_entries = [raw[indices[idx][0]+1:indices[idx][1]] for idx in range(len(indices))]
    all_txt = []

    for x, entry in enumerate(all_entries):
        empty = []
        for idx, item in enumerate(entry):
            if '<text>' in str(item):
                add_txt = entry[idx].replace('<text>', '')
                add_txt = add_txt.replace('</text>', '')
                add_txt = add_txt.replace('\n', '')
                empty.append(add_txt)
            else:
                pass
        all_txt.append(''.join(empty))
    return all_txt

text_db = get_raw_txt(ROOT + 'ml/data/nlm-chem-corpus/ALL/1253656_v1.xml')

custom_replace_transform = custom_replace([(r'\n', ''), (r'<', ''), (r'>', ''), (r'\t', '')])
preprocess = lambda x: (simple_space_split(custom_replace_transform(x)))

class PaperTextDataset(Dataset):
    def __init__ (self):
        self.data = {}
        self.embeddings = []
        self.length_axis = []
    
    def add_dataset(self, name, path, transformation):
        transformed_data = []
        for file in os.listdir(path):
            try:
                processed_txt = list(preprocess(transformation(path+'/'+file)))
                processed_txt = list(chain.from_iterable(processed_txt))
                processed_txt = [x.lower() for x in processed_txt if (x not in stopwords.words('english')) or (x not in ['a', 'it', 'they', 'I', 'you', 'since', 'because', 'thus', 'therefore', 'however', 'some'])]
                transformed_data.append(processed_txt[:510])
                self.length_axis.append(len(processed_txt[:510]))
            except:
                print(path+'/'+file)
                break

        # transformed_data = [preprocess(transformation(file)) for file in os.listdir(path)]
        self.data[name] = transformed_data
        return f"Added {name} to dataset."

    def __len__(self):
        return len(self.data)

    def __getitem__(self, name, idx):
        sample = self.data[name][idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def embed(self, tokenizer, model):
        self.embeddings = []
        for _, data in self.data.items():
            for text in data:
                with torch.no_grad():
                    inputs = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
                    outputs = model(inputs)
                    latent_output = outputs[0]
                    self.embeddings.append(latent_output)
        return self.embeddings


db = PaperTextDataset()
db.add_dataset('nlm-chem-corpus', ROOT+'ml/data/nlm-chem-corpus/ALL', get_raw_txt)

import json
import onnx
from transformers import BertModel, BertConfig, BertTokenizer, BertForPreTraining, load_tf_weights_in_bert

#use biobert main because it's the parameters loaded into the BERT Cased
#reference for BERT mode: https://github.com/google-research/bert

CONFIG = 'ml/models/biobert_v1.1_pubmed/bert_config.json'
WEIGHTS = 'ml/models/biobert_v1.1_pubmed/model.ckpt-1000000.index'
LOAD_MODEL = 'ml/models/biobert.pt'

def main(config_path, init_checkpoint_path, model_save_path):
    print('Configuring BERT model parameters ...')
    config = BertConfig().from_json_file(ROOT+config_path)
    print('Loading BERT model weights ...')
    model = BertModel.from_pretrained(ROOT+WEIGHTS, config=config, from_tf=True)
    print('Saving BioBert Model ...')
    torch.save(model.state_dict(), ROOT+model_save_path)
    print('Done!')
    return model

biobert = main(CONFIG, WEIGHTS, LOAD_MODEL)
tokenizer = BertTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed', do_lower_case=True)
biobert.eval()

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'load_dataset' from 'datasets' (unknown location)

# getting a lens eg. 

In [3]:
FILE = 'ml/data/export-dd.json'

db = pd.read_json(ROOT+FILE)
db.head()

,abstract,authors,chemicals,date_published,end_page,fields_of_study,funding,issue,keywords,lens_id,...,patent_citations_count,publication_type,references,scholarly_citations_count,source,source_urls,start_page,title,volume,year_published
0,"NLGN3 links oxytocin signaling, protein synthe...","[{'affiliations': [{'addresses': [], 'ids': []...",[],{'date': '2020-08-18T00:00:00+00:00'},None,"[{'name': 'Psychology'}, {'name': 'Prosocial b...",[],645,[],000-000-326-262-378,...,0,journal article,[{'lens_id': '077-677-839-990-811'}],0,"{'country': 'United States', 'issn': [{'type':...","[{'type': 'unknown', 'url': 'https://stke.scie...",None,A prosocial neuroligin,13,2020
1,Summary Females of many animal species behave ...,[{'affiliations': [{'addresses': [{'country_co...,"[{'mesh_id': 'D029721', 'registry_number': '0'...",{'date': '2009-02-26T00:00:00+00:00'},518,"[{'name': 'Internal medicine'}, {'name': 'Endo...",[],4,[],000-004-303-413-936,...,1,journal article,"[{'lens_id': '001-640-611-550-282'}, {'lens_id...",223,"{'country': 'United States', 'issn': [{'type':...","[{'type': 'html', 'url': 'https://www.ncbi.nlm...",511,Sensory neurons in the Drosophila genital trac...,61,2009
2,The enormous complexity of the central nervous...,[{'affiliations': [{'addresses': [{'country_co...,[],{'date': '2017-11-01T00:00:00+00:00'},571,"[{'name': 'Posttranslational modification'}, {...","[{'country': None, 'funding_id': None, 'org': ...",3,"[{'value': 'brain'}, {'value': 'central nervou...",000-004-360-715-324,...,0,journal article,"[{'lens_id': '000-005-937-882-552'}, {'lens_id...",50,"{'country': 'United States', 'issn': [{'type':...","[{'type': 'html', 'url': 'https://www.scienced...",558,A Primer on Concepts and Applications of Prote...,96,2017
3,T cell receptor (TCR) signaling is initiated a...,[{'affiliations': [{'addresses': [{'country_co...,"[{'mesh_id': 'D000199', 'registry_number': '0'...",{'date': None},127,"[{'name': 'Actin'}, {'name': 'Membrane protein...","[{'country': 'United States', 'funding_id': 'P...",1,[],000-009-038-775-698,...,7,journal article,"[{'lens_id': '000-290-175-402-908'}, {'lens_id...",767,"{'country': 'United States', 'issn': [{'type':...","[{'type': None, 'url': 'http://www.cell.com/im...",117,T cell receptor-proximal signals are sustained...,25,2006
4,None,[],[],{'date': '2001-03-27T00:00:00+00:00'},tw15,"[{'name': 'Blocking (statistics)'}, {'name': '...",[],75,[],000-009-723-468-418,...,0,None,[],0,"{'country': 'United States', 'issn': [{'type':...",[],tw15,Blocking a Survival Pathway,2001,2001


# Pipeline

In [14]:
# test BERN NER
import requests
from bs4 import BeautifulSoup
import requests
import re
from SPARQLWrapper import SPARQLWrapper, JSON

def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

text_eg = '''
Autophagy maintains tumour growth through circulating arginine. Autophagy captures intracellular components and delivers them to lysosomes, 
where they are degraded and recycled to sustain metabolism and to enable survival during starvation1-5. Acute, whole-body deletion of the 
essential autophagy gene Atg7 in adult mice causes a systemic metabolic defect that manifests as starvation intolerance and gradual loss of 
white adipose tissue, liver glycogen and muscle mass1. Cancer cells also benefit from autophagy.
'''
text = db['abstract'][1]

bern_res = [i for i in query_plain(text_eg)['annotations'] if i['prob'] > 0.5]
# print(bern_res[0])

# query for terms that have mesh ids and query for non-mesh id terms with their mentioned names -> end goal: build mesh mapping for all terms

sparql = SPARQLWrapper(
    "http://id.nlm.nih.gov/mesh/sparql"
)
sparql.setReturnFormat(JSON)

gen_funcs = {
    'sparql': lambda a: '''
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
        PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
        PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
        PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
        PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>

        SELECT ?EntryTerm
        FROM <http://id.nlm.nih.gov/mesh>
        WHERE {
        mesh:%s meshv:concept ?concept .
        ?concept meshv:term ?term .
        ?term rdfs:label ?EntryTerm
        } 
    '''%(a),
    'synonyms': lambda a: 'https://id.nlm.nih.gov/mesh/sparql?query=PREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0D%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0D%0APREFIX%20xsd%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0D%0APREFIX%20owl%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0APREFIX%20meshv%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2Fvocab%23%3E%0D%0APREFIX%20mesh%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F%3E%0D%0APREFIX%20mesh2015%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2015%2F%3E%0D%0APREFIX%20mesh2016%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2016%2F%3E%0D%0APREFIX%20mesh2017%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2017%2F%3E%0D%0A%0D%0ASELECT%20%3FEntryTerm%0D%0AFROM%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%3E%0D%0AWHERE%20%7B%0D%0A%20%20mesh%3A{}%20meshv%3Aconcept%20%3Fconcept%20.%0D%0A%20%20%3Fconcept%20meshv%3Aterm%20%3Fterm%20.%0D%0A%20%20%3Fterm%20rdfs%3Alabel%20%3FEntryTerm%0D%0A%7D%20%0D%0A%0D%0A%23Entry%20Terms%20for%20Ofloxacin%20(D015242)%0D%0A&format=JSON&limit=50&offset=0&inference=true'.format(a),
    'descriptor': lambda a: 'https://id.nlm.nih.gov/mesh/sparql?query=PREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0D%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0D%0APREFIX%20xsd%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0D%0APREFIX%20owl%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0APREFIX%20meshv%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2Fvocab%23%3E%0D%0APREFIX%20mesh%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F%3E%0D%0APREFIX%20mesh2015%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2015%2F%3E%0D%0APREFIX%20mesh2016%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2016%2F%3E%0D%0APREFIX%20mesh2017%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2017%2F%3E%0D%0A%0D%0A%20SELECT%20%3Fd%20%3FdName%0D%0A%20FROM%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%3E%0D%0A%20%0D%0A%20WHERE%20%7B%0D%0A%20%20%20%0D%0A%20%3Fd%20a%20meshv%3ADescriptor%20.%0D%0A%20%3Fd%20meshv%3Aactive%201%20.%0D%0A%20%3Fd%20rdfs%3Alabel%20%3FdName%20.%0D%0A%20FILTER(REGEX(%3FdName%2C%22{}%22%2C%22i%22))%0D%0A%20%0D%0A%20%7D%0D%0A%20ORDER%20BY%20%3Fd%0D%0A%0D%0A%23Find%20a%20set%20of%20descriptors%20and%20terms%20that%20contain%20%22infection.%22&format=JSON&limit=50&offset=0&inference=true'.format(a),
    'NCBIGene': lambda a: "https://pubchem.ncbi.nlm.nih.gov/rest/pug/gene/geneid/{}/summary/JSON".format(a),
    'NCBITaxon': lambda a: "https://pubchem.ncbi.nlm.nih.gov/rest/pug/taxonomy/taxid/{}/summary/JSON".format(a)

}

mesh_terms = {}
qualifier_terms ={}
concept_terms = {}

for term in bern_res:
    # print (term['prob']) 
    # print(term['id'][0])
    id_raw = term['id'][0]
    if 'mesh' in str(id_raw): 
        # print(term['id'])
        idx = 'D'+str(re.findall(r'\d+', id_raw)[0])
        mesh_res = requests.get(f'https://id.nlm.nih.gov/mesh/{idx}.json').json()
        synonyms = [i['EntryTerm']['value'] for i in requests.get(gen_funcs['synonyms'](idx)).json()['results']['bindings']]
        # print(mesh_res)
        # break
        mesh_terms[idx] = {
            'heading': mesh_res['label']['@value'],
            'qualifiers': [i[-6:] for i in mesh_res['allowableQualifier']] if 'allowableQualifier' in mesh_res.keys() else None,
            'concepts': [i[-6:] for i in mesh_res['concept']] if 'concept' in mesh_res.keys() else None,
            'pharmacological_actions': mesh_res['pharmacologicalAction'] if 'pharmacologicalAction' in mesh_res.keys() else None,
            'synonyms': synonyms if synonyms else None,
        }

    elif 'NCBI' in str(id_raw):
        print('NCBI', id_raw, term['mention'])
        if 'NCBIGene' in str(id_raw):
            gene_id = re.findall(r'\d+', id_raw)[0]
            ncbi_res = requests.get(gen_funcs['NCBIGene'](gene_id)).json()
            # print(ncbi_res)
        elif 'NCBITaxon' in str(id_raw):
            tax_id = re.findall(r'\d+', id_raw)[0]
            ncbi_res = requests.get(gen_funcs['NCBITaxon'](tax_id)).json()
            # print(ncbi_res)

    else:
        print('no mesh', id_raw, term['mention'])
        try:
            print(idx)
            sparql.setQuery(gen_funcs['sparql'](idx)) # not related to mesh mention query -> need to use lookup api 
            # print(sparql.queryString)
            
            ret = sparql.queryAndConvert()

            print('terms', ret)

        except Exception as e:
            print('error', e)
        

print(mesh_terms['D009369'])
# print([i['mention'] for i in bern_res])

NER_mesh = [i['id'] for i  in bern_res]
print(NER_mesh)

# mesh_ids = [i['mesh_id'] for i in db['mesh_terms'][1]]
# print(mesh_ids)

# types of tags: NCBITaxon, NCBIGene, CUI-less, MESH
#eg. sparlql url https://id.nlm.nih.gov/mesh/sparql?query=PREFIX%20rdf%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0D%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0D%0APREFIX%20xsd%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0D%0APREFIX%20owl%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2002%2F07%2Fowl%23%3E%0D%0APREFIX%20meshv%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2Fvocab%23%3E%0D%0APREFIX%20mesh%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F%3E%0D%0APREFIX%20mesh2015%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2015%2F%3E%0D%0APREFIX%20mesh2016%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2016%2F%3E%0D%0APREFIX%20mesh2017%3A%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%2F2017%2F%3E%0D%0A%0D%0ASELECT%20%3FEntryTerm%0D%0AFROM%20%3Chttp%3A%2F%2Fid.nlm.nih.gov%2Fmesh%3E%0D%0AWHERE%20%7B%0D%0A%20%20mesh%3A{}%20meshv%3Aconcept%20%3Fconcept%20.%0D%0A%20%20%3Fconcept%20meshv%3Aterm%20%3Fterm%20.%0D%0A%20%20%3Fterm%20rdfs%3Alabel%20%3FEntryTerm%0D%0A%7D%20%0D%0A%0D%0A%23Entry%20Terms%20for%20Ofloxacin%20(D015242)%0D%0A&format=JSON&limit=50&offset=0&inference=true

no mesh CUI-less essential autophagy gene
D001120
terms {'head': {'vars': ['EntryTerm']}, 'results': {'bindings': []}}
NCBI NCBIGene:8408 autophagy gene
NCBI NCBIGene:10533 Atg7
NCBI NCBITaxon:10095 mice
no mesh CUI-less Cancer cells
D013217
terms {'head': {'vars': ['EntryTerm']}, 'results': {'bindings': []}}
{'heading': 'Neoplasms', 'qualifiers': ['000175', '000235', '000150', '000098', '000451', '000648', '000737', '000266', '000382', '000188', '000601', '000201', '000981', '000821', '000628', '000534', '000178', '000523', '000401', '000473', '000208', '000139', '000097', '000196', '000134', '000503', '000145', '000453', '000469', '000532', '000151', '000209', '000378', '000517', '000276', '000652', '000191', '000662'], 'concepts': ['014586', '014584'], 'pharmacological_actions': None, 'synonyms': ['Neoplasm, Malignant', 'Neoplasms, Malignant', 'Cancers', 'Neoplasm, Benign', 'Neoplasias', 'Malignancies', 'Malignant Neoplasms', 'Cancer', 'Neoplasms, Benign', 'Benign Neoplasms', 'Tumor

In [96]:
import ast

print(ast.literal_eval(ret.decode('utf-8')))

# idx = re.findall(r'\d+', str(NER_mesh[0]))[0]
# database_type = set(str(NER_mesh[0])).difference(idx)

# html = requests.get('https://www.ncbi.nlm.nih.gov/taxonomy/7227')

SyntaxError: invalid syntax (<unknown>, line 1)

In [4]:
# abstract, chemicals, fields_of_study, keywords, lens_id, mesh_terms, references, title -> with BERN we get more specificity on keywords 
# NER type can be replaced with MeSH qualifiers 
# NER types: Gene, Disease, Chemical, Species, Mutation, Cell Line, Cell Type, DNA, and RNA

class Node:
    def __init__(self, doi):
        print("deez nuts")
        

node_props = {
    '{doi}': {
        'title': str,
        'ref_list': [], #doi
        'mesh_terms': {
            'Gene': [], # mesh ids
            'Disease': [] #mesh ids
            }, 
        'authors': [],
        'keywords': [],
        'publication_type': '',
        },
        
}

mesh_props = {
    '{mesh_id}': {
        # 'mesh_tree': {},
        'heading': str,
        'synonyms': [],
        'qualifiers': {},
        'concepts': {},
        'pharm_actions': {},

    }
}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05362208-35d3-4fa0-ada7-4013d10444bf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>